In [1]:
import pandas as pd
import numpy as np

In [2]:
import re
def pre_process(text):
  url_pattern="https?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F\][0-9a-fA-F]))+"
  non_ascii_pattern="[^\x00-\x7F]+"
  url_matches=re.findall(url_pattern,text)
  non_ascii_matches=re.findall(non_ascii_pattern,text)
  text=text.lower()
  text=text.replace("\n"," ")
  text=text.strip()
  if url_matches:
    for url in url_matches:
      text=text.replace(url,"")
  if non_ascii_matches:
    for non_ascii in non_ascii_matches:
      text=text.replace(non_ascii,"")
  text=text.strip()
  if len(text.split())<=2:
    return np.nan
  return text

<>:3: SyntaxWarning: invalid escape sequence '\]'
<>:3: SyntaxWarning: invalid escape sequence '\]'
/var/folders/hj/pb36nbg93kz0c0dn51ck371w0000gq/T/ipykernel_290/875611287.py:3: SyntaxWarning: invalid escape sequence '\]'
  url_pattern="https?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F\][0-9a-fA-F]))+"


In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
lemm=WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/keertan.patro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/keertan.patro/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
df=pd.read_csv("https://raw.githubusercontent.com/Himanshu-1703/reddit-sentiment-analysis/refs/heads/main/data/reddit.csv")
df.dropna(inplace=True)

In [5]:
df.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [7]:
stopwords=[word for word in stopwords.words('english') if word not in ('not','but','however','no','yet')]

In [8]:
def remove_stopwords(text):
  new_text=""
  for word in text.split():
    if word not in stopwords:
      word=lemm.lemmatize(word)
      new_text+=word+" "
  return new_text.strip()

In [9]:
df['new_clean_comment']=df['clean_comment'].apply(pre_process)
df.dropna(inplace=True)
df['processed_comment']=df['new_clean_comment'].apply(remove_stopwords)

In [10]:
def get_max_length(df):
  max_length=-float('inf')
  for text in df['processed_comment']:
    text_len=len(text.split())
    if text_len>max_length:
      max_length=text_len
  return max_length
max_length=get_max_length(df)

In [11]:
max_length

893

In [12]:
df['new_category']=df['category'].apply(lambda x: x if x in (1,0) else 2 )

In [13]:
df['new_category'].value_counts()

,count
new_category,
1,15446
0,11005
2,8066


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import dataset,DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [15]:
tokenizer=Tokenizer(num_words=2000,lower=True)
tokenizer.fit_on_texts(df['processed_comment'])

In [16]:
vocab=tokenizer.word_index
len(vocab)
max_len=600
X=tokenizer.texts_to_sequences(df['processed_comment'])
X=pad_sequences(X,maxlen=max_len)
y=np.array(df['new_category'])

In [17]:
device="cuda" if torch.cuda.is_available else "cpu"

In [18]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True,stratify=y)
X_train=torch.tensor(X_train,dtype=torch.long).to(device)
X_test=torch.tensor(X_test,dtype=torch.long).to(device)
y_train=torch.tensor(y_train,dtype=torch.long).to(device)
y_test=torch.tensor(y_test,dtype=torch.long).to(device)
train_dataset=TensorDataset(X_train,y_train)
test_dataset=TensorDataset(X_test,y_test)

In [19]:
batch_size=4
output_dim=3
train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_dataloader=DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [20]:
class Lstm_model(nn.Module):
  def __init__(self,vocab_size,embedding_dim,hidden_dim):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,embedding_dim)
    self.lstm=nn.LSTM(embedding_dim,hidden_dim,batch_first=True)
    self.linear=nn.Linear(hidden_dim,output_dim)

  def forward(self,x):
    x=self.embedding(x)
    output,hidden=self.lstm(x)
    x=hidden[-1]
    y=self.linear(x)
    return y

In [21]:
print(torch.version.cuda)

12.6


In [22]:
model=Lstm_model(len(vocab),128,64).to(device)

In [23]:
model

Lstm_model(
  (embedding): Embedding(47894, 128)
  (lstm): LSTM(128, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=3, bias=True)
)

In [24]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.0001)

In [25]:
from tqdm import tqdm

In [29]:
model.train()
for epoch in range(25):
  total_loss=0
  for x,y in tqdm(train_dataloader):
    y_pred=model(x)
    y_pred=y_pred.squeeze()
    batch_size=y.shape[0]
    y_pred=y_pred.view(batch_size,output_dim)
    loss=criterion(y_pred,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss+=loss.item()
  print(f"Loss for epoch-{epoch} is", total_loss)

100%|██████████| 6904/6904 [00:29<00:00, 230.74it/s]


Loss for epoch-0 is 6083.170913055539


100%|██████████| 6904/6904 [00:29<00:00, 235.48it/s]


Loss for epoch-1 is 4942.615423664451


100%|██████████| 6904/6904 [00:29<00:00, 235.18it/s]


...

Loss for epoch-22 is 716.3786943701031


100%|██████████| 6904/6904 [00:29<00:00, 236.98it/s]


Loss for epoch-23 is 662.1217200150322


100%|██████████| 6904/6904 [00:29<00:00, 236.53it/s]

Loss for epoch-24 is 603.8883051154452


In [30]:
def evaluate_model(model,test_dataloader):
  model.eval()
  y_pred_all=[]
  y_true_all=[]
  for x,y in test_dataloader:
    y_pred=model(x)
    y_pred=torch.max(y_pred,-1).indices.view(y.shape[0])
    if y_pred.device.type=="cuda":
      y_pred=y_pred.to("cpu").tolist()
      y=y.to("cpu").tolist()
      y_pred_all.extend(y_pred)
      y_true_all.extend(y)
    elif y_pred.device.type=="cpu":
      y_pred=y_pred.tolist()[0]
      y_pred_all.extend(y_pred)
      y=y.tolist()
      y_true_all.extend(y)
  report=classification_report(y_true_all,y_pred_all,output_dict=True)
  return report,y_true_all,y_pred_all

In [25]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 8.7 MB/s eta 0:00:00


In [26]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 703.4/703.4 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 16.2 MB/s eta 0:00:00


In [27]:
! pip install awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 39.7 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9.1
    Uninstalling rsa-4.9.1:
      Successfully uninstalled rsa-4.9.1
  Attempting uninstall: docutils
    Found existing installation: docutils 0.21.2
    Uninstalling docutils-0.21.2:
      Successfully uninstalled docutils-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.2.3 requires docutils<0.22,>=0.20, but you have docutils 0.19 which is incompatible.


In [31]:
from sklearn.metrics import accuracy_score,recall_score,classification_report
import mlflow
import boto3
mlflow.set_tracking_uri("http://ec2-54-211-106-118.compute-1.amazonaws.com:5000/")

In [32]:
def store_best_model_lstm(model,test_dataloader):
  with mlflow.start_run():
    mlflow.set_tag("mlflow.runName", "LSTM")
    mlflow.set_tag("experiment_type", "algorithm_comparison")
    class_report,y_true_all,y_pred_all=evaluate_model(model,test_dataloader)
    print("class report:",class_report)
    for metric in class_report:
      mlflow.log_param("Model","LSTM")
      for metric in class_report:
        if type(class_report[metric])==dict:
          for key in class_report[metric]:
            if key!='support':
              mlflow.log_metric(f"{metric}_{key}",class_report[metric][key])
        else:
          mlflow.log_metric(metric,class_report[metric])
    mlflow.pytorch.log_model(model, "Lstm model")

In [33]:
store_best_model_lstm(model,test_dataloader)

class report: {'0': {'precision': 0.8048128342245989, 'recall': 0.8205361199454794, 'f1-score': 0.8125984251968504, 'support': 2201.0}, '1': {'precision': 0.7687981053878035, 'recall': 0.8404530744336569, 'f1-score': 0.803030303030303, 'support': 3090.0}, '2': {'precision': 0.7238689547581904, 'recall': 0.5753254804711717, 'f1-score': 0.6411053540587219, 'support': 1613.0}, 'accuracy': 0.7721610660486674, 'macro avg': {'precision': 0.7658266314568642, 'recall': 0.7454382249501026, 'f1-score': 0.7522446940952917, 'support': 6904.0}, 'weighted avg': {'precision': 0.7697827082563174, 'recall': 0.7721610660486674, 'f1-score': 0.7682496677750034, 'support': 6904.0}}


2025/08/20 13:01:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/20 13:02:07 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torch==2.8.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/08/20 13:02:11 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torch==2.8.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/08/20 13:02:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter whe

🏃 View run LSTM at: http://ec2-54-211-106-118.compute-1.amazonaws.com:5000/#/experiments/0/runs/17165c9c2fa74fe09e99c03a53d415fe
🧪 View experiment at: http://ec2-54-211-106-118.compute-1.amazonaws.com:5000/#/experiments/0
